In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
#Leo la data proveniente de un csv
path = 'datos/installs.csv'
installs = pd.read_csv(path)

/home/santiago/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (4,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Damos formato a la fecha.
installs['timestamp_format'] = pd.to_datetime(installs['created'])
# Agrego columna fecha
installs['date_timestamp'] = installs['timestamp_format'].dt.date
# Agrego columna numero dia.
installs['day'] = installs['timestamp_format'].dt.day

In [4]:
# ventanas de 3 dias
installs18_20 = installs[(installs['day'] >= 18) & (installs['day'] < 21)]
installs19_21 = installs[(installs['day'] >= 19) & (installs['day'] < 22)]
installs20_22 = installs[(installs['day'] >= 20) & (installs['day'] < 23)]
installs21_23 = installs[(installs['day'] >= 21) & (installs['day'] < 24)]
installs22_24 = installs[(installs['day'] >= 22) & (installs['day'] < 25)]
installs23_25 = installs[(installs['day'] >= 23) & (installs['day'] < 26)]
installs24_26 = installs[(installs['day'] >= 24) & (installs['day'] < 27)]

In [5]:
# Agregamos variable a predecir a los dataframes
installs18_20['base_time'] = datetime.datetime(2019, 4, 18)
installs18_20['time_to_install'] = (installs18_20['timestamp_format'] - installs18_20['base_time']) / np.timedelta64(1, 's')

installs19_21['base_time'] = datetime.datetime(2019, 4, 19)
installs19_21['time_to_install'] = (installs19_21['timestamp_format'] - installs19_21['base_time']) / np.timedelta64(1, 's')

installs20_22['base_time'] = datetime.datetime(2019, 4, 20)
installs20_22['time_to_install'] = (installs20_22['timestamp_format'] - installs20_22['base_time']) / np.timedelta64(1, 's')

installs21_23['base_time'] = datetime.datetime(2019, 4, 21)
installs21_23['time_to_install'] = (installs21_23['timestamp_format'] - installs21_23['base_time']) / np.timedelta64(1, 's')

installs22_24['base_time'] = datetime.datetime(2019, 4, 22)
installs22_24['time_to_install'] = (installs22_24['timestamp_format'] - installs22_24['base_time']) / np.timedelta64(1, 's')

installs23_25['base_time'] = datetime.datetime(2019, 4, 23)
installs23_25['time_to_install'] = (installs23_25['timestamp_format'] - installs23_25['base_time']) / np.timedelta64(1, 's')

installs24_26['base_time'] = datetime.datetime(2019, 4, 24)
installs24_26['time_to_install'] = (installs24_26['timestamp_format'] - installs24_26['base_time']) / np.timedelta64(1, 's')

/home/santiago/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/santiago/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/santiago/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

# Feature Engineering

## Cantidad de Instalaciones

In [122]:
final = installs18_20.groupby('ref_hash')['created'].count().reset_index()
final.columns = ['ref_hash', 'cant_installs']
final2 = installs21_23.groupby('ref_hash')['created'].count().reset_index()
final2.columns = ['ref_hash', 'cant_installs']

## Cantidad de Instalaciones con wifi

In [123]:
installs18_20['wifi'].fillna(0, inplace= True)
installs18_20['wifi'].astype('int64', inplace= True)
installs_con_wifi = installs18_20.groupby('ref_hash').agg({'wifi': 'sum'}).reset_index()
installs_con_wifi.columns = ['ref_hash', 'cant_wifi_installs']

final = final.merge(installs_con_wifi, on='ref_hash', how='left')
final['cant_wifi_installs'].fillna(0, inplace= True)

In [124]:
installs21_23['wifi'].fillna(0, inplace= True)
installs21_23['wifi'].astype('int64', inplace= True)
installs_con_wifi2 = installs21_23.groupby('ref_hash').agg({'wifi': 'sum'}).reset_index()
installs_con_wifi2.columns = ['ref_hash', 'cant_wifi_installs']

final2 = final2.merge(installs_con_wifi2, on='ref_hash', how='left')
final2['cant_wifi_installs'].fillna(0, inplace= True)

## Marca de celular del user

In [125]:
brand_user = installs18_20.groupby('ref_hash').agg({'device_brand': 'first'}).reset_index()
brand_user.columns = ['ref_hash', 'device_brand']

final = final.merge(brand_user, on='ref_hash', how='left')
final['device_brand'].fillna(0, inplace= True)

In [126]:
brand_user2 = installs21_23.groupby('ref_hash').agg({'device_brand': 'first'}).reset_index()
brand_user2.columns = ['ref_hash', 'device_brand']

final2 = final2.merge(brand_user2, on='ref_hash', how='left')
final2['device_brand'].fillna(0, inplace= True)

## Modelo de celular del user

In [127]:
modelo_user = installs18_20.groupby('ref_hash').agg({'device_model': 'first'}).reset_index()
modelo_user.columns = ['ref_hash', 'device_model']

final = final.merge(modelo_user, on='ref_hash', how='left')
final['device_model'].fillna(0, inplace= True)

In [128]:
modelo_user2 = installs21_23.groupby('ref_hash').agg({'device_model': 'first'}).reset_index()
modelo_user2.columns = ['ref_hash', 'device_model']

final2 = final2.merge(modelo_user2, on='ref_hash', how='left')
final2['device_model'].fillna(0, inplace= True)

## Id de la aplicacion instalada

In [129]:
app_id = installs18_20.groupby('ref_hash').agg({'application_id': 'last'}).reset_index()
app_id.columns = ['ref_hash', 'application_id']

final = final.merge(app_id, on='ref_hash', how='left')
final['application_id'].fillna(0, inplace= True)

In [130]:
app_id2 = installs21_23.groupby('ref_hash').agg({'application_id': 'last'}).reset_index()
app_id2.columns = ['ref_hash', 'application_id']

final2 = final2.merge(app_id2, on='ref_hash', how='left')
final2['application_id'].fillna(0, inplace= True)

## User agent

In [131]:
user_agent = installs18_20.groupby('ref_hash').agg({'user_agent': 'last'}).reset_index()
user_agent.columns = ['ref_hash', 'user_agent']

final = final.merge(user_agent, on='ref_hash', how='left')

In [132]:
user_agent2 = installs21_23.groupby('ref_hash').agg({'user_agent': 'last'}).reset_index()
user_agent2.columns = ['ref_hash', 'user_agent']

final2 = final2.merge(user_agent2, on='ref_hash', how='left')

## Device Language

In [133]:
device_language = installs18_20.groupby('ref_hash').agg({'device_language': 'last'}).reset_index()
device_language.columns = ['ref_hash', 'device_language']

final = final.merge(device_language, on='ref_hash', how='left')

In [134]:
device_language2 = installs21_23.groupby('ref_hash').agg({'device_language': 'last'}).reset_index()
device_language2.columns = ['ref_hash', 'device_language']

final2 = final2.merge(device_language2, on='ref_hash', how='left')

## Attributed

In [135]:
installs18_20['attributed'].astype('int64', inplace= True)
attributed = installs18_20.groupby('ref_hash').agg({'attributed': 'max'}).reset_index()
attributed.columns = ['ref_hash', 'attributed']

final = final.merge(attributed, on='ref_hash', how='left')

In [136]:
installs21_23['attributed'].astype('int64', inplace= True)
attributed2 = installs21_23.groupby('ref_hash').agg({'attributed': 'max'}).reset_index()
attributed2.columns = ['ref_hash', 'attributed']

final2 = final2.merge(attributed2, on='ref_hash', how='left')

## TimeToInstall -> variable a predecir

In [137]:
time_to_install = installs18_20.groupby('ref_hash').agg({'time_to_install': 'mean'}).reset_index()
time_to_install.columns = ['ref_hash', 'time_to_install']

# 3 dias son 259200 segundos
final = final.merge(time_to_install, on='ref_hash', how='left')
final['time_to_install'].fillna(259200, inplace= True)

In [138]:
time_to_install2 = installs21_23.groupby('ref_hash').agg({'time_to_install': 'mean'}).reset_index()
time_to_install2.columns = ['ref_hash', 'time_to_install']

# 3 dias son 259200 segundos
final2 = final2.merge(time_to_install2, on='ref_hash', how='left')
final2['time_to_install'].fillna(259200, inplace= True)

In [139]:
final.head()

,ref_hash,cant_installs,cant_wifi_installs,device_brand,device_model,application_id,user_agent,device_language,attributed,time_to_install
0,41863526108385,4,0,6.115026e+18,1.658417e+18,121,Dalvik/2.1.0 (Linux; U; Android 8.0.0; LDN-LX3...,6.977049e+18,False,76472.69225
1,135153013040192,1,0,0.000000e+00,4.981016e+18,155,NaN,3.301378e+18,False,192980.25800
2,186034136943920,1,0,0.000000e+00,0.000000e+00,210,NaN,NaN,False,60042.83900
3,365882020742330,1,False,3.083059e+17,1.805456e+18,302,Dalvik/2.1.0 (Linux; U; Android 8.0.0; SM-G570...,6.977049e+18,False,96156.50800
4,519199987760489,1,True,8.461030e+18,1.295045e+18,187,Dalvik/2.1.0 (Linux; U; Android 7.1.1; H3321 B...,6.977049e+18,False,92860.47500


In [140]:
final2.head()

,ref_hash,cant_installs,cant_wifi_installs,device_brand,device_model,application_id,user_agent,device_language,attributed,time_to_install
0,40621409780134,1,0,0.000000e+00,2.019322e+18,77,NaN,3.301378e+18,False,69467.657
1,161514654074162,1,True,2.208835e+18,6.684966e+18,65,Dalvik/2.1.0 (Linux; U; Android 9; moto g(6) p...,6.977049e+18,False,152024.805
2,168103949904656,1,0,0.000000e+00,1.816356e+18,155,NaN,3.301378e+18,False,86973.188
3,283297668933729,1,0,0.000000e+00,1.805456e+18,210,Dalvik/2.1.0 (Linux; U; Android 7.0; SM-G570M ...,3.301378e+18,False,135729.476
4,345999128501141,1,0,0.000000e+00,1.709140e+17,155,NaN,3.301378e+18,False,15578.532


## Join de targets con features

In [141]:
targets = pd.read_csv('datos/target_competencia_ids.csv')

In [142]:
final['ref_hash'] = final['ref_hash'].astype(str) + '_sc'
final['ref_hash'] = final['ref_hash'].astype('object')

merged_18_20_targets = targets.merge(final, on='ref_hash', how='left').drop('obj', axis=1)
merged_18_20_targets = merged_18_20_targets.loc[merged_18_20_targets['ref_hash'].str.contains('_sc'), :]

# 3 dias son 259200 segundos
merged_18_20_targets['time_to_install'] = merged_18_20_targets['time_to_install'].fillna(259200)

In [143]:
final2['ref_hash'] = final2['ref_hash'].astype(str) + '_sc'
final2['ref_hash'] = final2['ref_hash'].astype('object')

merged_21_23_targets = targets.merge(final2, on='ref_hash', how='left').drop('obj', axis=1)
merged_21_23_targets = merged_21_23_targets.loc[merged_21_23_targets['ref_hash'].str.contains('_sc'), :]

# 3 dias son 259200 segundos
merged_21_23_targets['time_to_install'] = merged_21_23_targets['time_to_install'].fillna(259200)

In [144]:
merged_18_20_targets

,ref_hash,cant_installs,cant_wifi_installs,device_brand,device_model,application_id,user_agent,device_language,attributed,time_to_install
0,1000169251625791246_sc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,259200.000000
2,1000395625957344683_sc,2.0,2,6.115026e+18,8.058599e+17,121.0,Dalvik/2.1.0 (Linux; U; Android 8.0.0; RNE-L03...,6.977049e+18,False,194366.897000
4,1003027494996471685_sc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,259200.000000
6,1006670001679961544_sc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,259200.000000
8,1007573308966476713_sc,1.0,0,0.000000e+00,1.327864e+18,94.0,NaN,3.301378e+18,False,27164.992000
10,1010070503877148763_sc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,259200.000000
12,1010265377387765028_sc,2.0,1,6.115026e+18,1.670346e+18,94.0,Dalvik/2.1.0 (Linux; U; Android 8.1.0; JKM-LX3...,3.301378e+18,False,96639.972000
14,1010531372912327058_sc,1.0,True,3.083059e+17,3.017401e+18,65.0,Dalvik/2.1.0 (Linux; U; Android 6.0.1; SM-G610...,6.977049e+18,False,158989.897000
16,1011610998357271358_sc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,259200.000000
18,1013543838965040946_sc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,259200.000000


In [145]:
merged_21_23_targets

,ref_hash,cant_installs,cant_wifi_installs,device_brand,device_model,application_id,user_agent,device_language,attributed,time_to_install
0,1000169251625791246_sc,1.0,0,0.000000e+00,1.805456e+18,122.0,NaN,3.301378e+18,False,29646.1780
2,1000395625957344683_sc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,259200.0000
4,1003027494996471685_sc,2.0,0,6.115026e+18,8.590181e+18,14.0,NaN,3.301378e+18,False,247044.3875
6,1006670001679961544_sc,2.0,0,0.000000e+00,6.794880e+18,68.0,Grability/17420 CFNetwork/978.0.7 Darwin/18.5.0,8.441417e+18,False,4844.0530
8,1007573308966476713_sc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,259200.0000
10,1010070503877148763_sc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,259200.0000
12,1010265377387765028_sc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,259200.0000
14,1010531372912327058_sc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,259200.0000
16,1011610998357271358_sc,2.0,0,3.083059e+17,5.572220e+18,14.0,NaN,3.301378e+18,False,5657.6230
18,1013543838965040946_sc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,259200.0000


In [146]:
merged_18_20_targets.to_csv('test_and_train_installs/data_18_20.csv', index=False)
merged_21_23_targets.to_csv('test_and_train_installs/data_21_23.csv', index=False)